In [ ]:
# Import relevant functions
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
from pyspark import broadcast, SparkContext
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover, OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer, Bucketizer
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.functions import vector_to_array
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.mllib.util import MLUtils
import re
import scipy
import string
import sys

In [ ]:
# Build the spark session
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [ ]:
spark.version

In [ ]:
sc = spark.sparkContext

## Load data from csv, strip to required size then save to parquet

In [ ]:
# Load all submission csv files
data = spark.read.format('csv').options(header ='true').load(f"./data/submissions/*.csv")

In [ ]:
print(data.printSchema())
data.count()

In [ ]:
data.head(5)

In [ ]:
# Only keep the needed columns and remove duplicate rows
df = data.select('id','title','domain','subreddit_id','created_utc','num_comments')
df = df.distinct()
df.count()

In [ ]:
# Fill nulls in the num_comments field with zero
df = df.na.fill(value=0,subset=["num_comments"])

In [ ]:
df.show(5)

In [ ]:
# Remove punctuation and numbers from titles
df = df.withColumn("title", F.regexp_replace(F.col("title"), '[^\sa-zA-Z]', ''))
df.show(5)

## Prepare titles and do topics modelling

In [ ]:
tokenizer = Tokenizer(inputCol="title", outputCol="words")
wordsDataFrame = tokenizer.transform(df)

In [ ]:
wordsDataFrame.show()

In [ ]:
stop_words =StopWordsRemover.loadDefaultStopWords("english")
stop_words = stop_words + ['a','i']

In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = stop_words)
wordsDataFrame = remover.transform(wordsDataFrame)

In [ ]:
wordsDataFrame.show(truncate = 50)

In [ ]:
cv = CountVectorizer(inputCol="filtered", outputCol="vectors")
cvmodel = cv.fit(wordsDataFrame)

#save the countvectorizer to apply to new texts
cvmodel.write().overwrite().save('count_vectorizer_model')
# When need to relaod this model elsewhere use : loadedModel = CountVectorizerModel.load('count_vectorizer_model')

df_vect = cvmodel.transform(wordsDataFrame)
basics = df_vect.select('vectors', 'id')
basics.show()

In [ ]:
basics.write.mode("overwrite").parquet('lda_basics')

In [ ]:
basics= spark.read.parquet('lda_basics')

In [ ]:
# Create the LDA model and fit it
num_topics = 20
lda = LDA(featuresCol='vectors',k=num_topics, seed=42)
#Train the LDA model
lda_model = lda.fit(basics)

# Save teh model to be applied later in stream pipeline
lda_model.write().overwrite().save('lda_distributed_model')
# When need to relaod this model elsewhere use : sameModel = DistributedLDAModel.load('lda_distributed_model')

In [ ]:
# See the results of modelling
ldatopics = lda_model.describeTopics(10)
ldatopics.show(truncate = 50)

In [ ]:
# For mapping words to the model term indices, first collect the words used in the input vectors
vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab) # saves the words to a broadcast variable for use in mapping
print("Number of words in word vectors: " + str(len(vocab)))
print("Example last words: ")
vocab[-6:-1]

In [ ]:
# Now match input words to terms in model output topics
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])

    return words

udf_map_termID_to_Word = F.udf(map_termID_to_Word , ArrayType(StringType()))

ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))

In [ ]:
ldatopics_mapped.show(truncate=False)

In [ ]:
# Use the model to allocate topic weightings for each title
indiv = lda_model.transform(basics)

In [ ]:
indiv.show()

## Build features and target dataset

In [ ]:
# convert epoch time into datetime, but keep epoch time as id
df= df.withColumn('timestamp', F.from_unixtime(df.created_utc,'yyyy-MM-dd HH:mm:ss')) 

In [ ]:
df = df.withColumn("hour", F.hour(F.col("timestamp"))).withColumn("day", F.dayofweek(F.col("timestamp")))
df = df.withColumn("hour", F.col("hour").astype(StringType())).withColumn("day", F.col("day").astype(StringType()))

In [ ]:
df.show(truncate=60)

In [ ]:
df.printSchema()

In [ ]:
# Create features df by combining topic distributions for each title, with the other features of domain, hour and day
temp = indiv.select('topicDistribution')
# And a list of sequential column titles
X_titles = [f'T_{i}' for i in range(1, num_topics +1)]
X_titles = ['id'] + X_titles # adding the id at start

In [ ]:
temp = indiv.withColumn("T_", vector_to_array("topicDistribution")).select(["id"] + [F.col("T_")[i] for i in range(num_topics)]).drop('vectors', 'topicDistribution')
temp = temp.toDF(*X_titles)
temp.show()

In [ ]:
rf_full = temp.join(df, on = 'id', how= 'left').drop('id','subreddit_id','title','timestamp','created_utc')

In [ ]:
rf_full.printSchema()

In [ ]:
rf_full.count()

In [ ]:
# Convert the number of comments into numerical and create a grouped categorical
rf_full = rf_full.withColumn("num_comments", F.col("num_comments").astype(IntegerType()))
bucketizer = Bucketizer(splitsArray=[[0,5,25,float("inf")]],inputCols=["num_comments"], outputCols=["group_comments"])
rf_full = bucketizer.setHandleInvalid("keep").transform(rf_full)
rf_full = rf_full.drop('num_comments')
rf_full.show(20)

In [ ]:
rf_full.printSchema()

In [ ]:
# Drop rows where there are null values, to avaoid probelms with modelling
rf_full = rf_full.na.drop("any")

In [ ]:
# Check all gone
print(rf_full.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in rf_full.columns]).show()) 

In [ ]:
rf_full.write.mode("overwrite").parquet('rf_full')

In [ ]:
rf_full = spark.read.parquet('rf_full')

## Now build the ML model

In [ ]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [ ]:
# Need to one-hot-encode the domain, hour and day columns as they are categorical
#Create variable for all categorical columns
cat_cols = ['domain','hour','day']
#Create and empty list and put the string indexed and one-hot-encoded transformations into it
stages = []
for cat_col in cat_cols:
    col_indexer = StringIndexer(inputCol=cat_col, outputCol=f"{cat_col}_ind",handleInvalid='skip')
    col_encoder = OneHotEncoder(inputCols=[f"{cat_col}_ind"], outputCols=[f"{cat_col}_ohe"])
    stages += [col_indexer, col_encoder]

In [ ]:
# Split data into train and test sets
train, test = rf_full.randomSplit([0.8, 0.2], seed=42)

# Seperate out the features from the data
X = rf_full.drop('group_comments')
y= rf_full.select('group_comments')

# Make list of numeric columns (here that is all columns in features not categorical)
features_cols = X.columns
num_cols = [x for x in features_cols if x not in cat_cols]

In [ ]:
# Change title of one not encoded categoricals
cat_cols_ohe = [f"{cat_col}_ohe" for cat_col in cat_cols]

In [ ]:
# Build all features into one vetor
assembler = VectorAssembler(inputCols=cat_cols_ohe + num_cols, outputCol="features")
stages += [assembler]

In [ ]:
# Specify the model core and target variable
rf = RandomForestClassifier(labelCol="group_comments",featuresCol="features")
stages += [rf]

In [ ]:
# Populate the pipeline
pipeline = Pipeline(stages=stages)

In [ ]:
# Fit the model to training set via pipeline
from datetime import datetime 
start_time = datetime.now() 

pipeline_model = pipeline.fit(train)

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

In [ ]:
# Save the model for later use
pipeline_model.write().overwrite().save('pipeline_model')

In [ ]:
# Test the model
test_preds = pipeline_model.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol=rf.getLabelCol(), predictionCol=rf.getPredictionCol(), metricName="accuracy")
accuracy = evaluator.evaluate(test_preds)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
test_preds.select('prediction').show(5)

In [ ]:
preds_and_labels = test_preds.select(['prediction','group_comments']).withColumn('label', F.col('group_comments').cast(FloatType())).orderBy('prediction')

In [ ]:
# Select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

In [ ]:
spark.stop()